In [2]:
import numpy as np
import pandas as pd

In [7]:
df_h = pd.read_csv('health_data.csv')
print(df_h.head())


   age  restbps  chol  category
0   26      109   243         0
1   27      106   156         0
2   28      107   225         0
3   27      105   277         0
4   30       96   221         0


In [25]:
array = df_h.to_numpy()
print(array.shape)
X = array[:, :3]
y = array[:, 3]
print(X, y)

(700, 4)
[[ 26 109 243]
 [ 27 106 156]
 [ 28 107 225]
 ...
 [ 46 121 219]
 [ 43 119 239]
 [ 42 132 251]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [8]:
# df_norm = normalize_data(df, ['category'])
# df_norm

In [9]:
class KNN():
    def __init__(self, k, traindf, testdf):
        self.k = k
        self.traindf = self.normalize_data(traindf, ['category'])
        self.testdf = self.normalize_data(testdf, ['category'])

    
    def classify(self):
        df_result =  self.testdf.copy()
        class_lab = np.zeros(self.testdf.shape[0])
        for i in range(self.testdf.shape[0]):
            class_lab[i] = self.get_class(self.testdf.iloc[i])
        
        df_result['classified_label'] = class_lab.tolist()
        return df_result
        
    def get_class(self, datapoint):
        dist_train = self.traindf.copy()
        dist = np.zeros(self.traindf.shape[0])
        for i in range(self.traindf.shape[0]):
            dist[i] = self.get_dist(datapoint, self.traindf.iloc[i])
        dist_train['dist'] = dist.tolist()
        dist_train = dist_train.sort_values('dist', ascending = True)
        k_nearest = dist_train.head(k)
        numones = k_nearest['category'].sum()
        if(numones>=k_nearest.shape[0]):
            label = 1
        else:
            label = 0
        return label   
    
    def normalize_data(self, df, no_change_column_list):
        norm_df = (df - df.min())/(df.max() - df.min())
        for col in no_change_column_list:
            norm_df[col] = df[col]
#         print('-----', norm_df)
        return norm_df

    def get_dist(self, x1, x2):
        return np.linalg.norm((x1-x2)[:-1])
        

In [10]:
def KNNclassify(k, traindf, testdf):
    knn = KNN(k, train_df, test_df)
    classified_test = knn.classify()
    return classified_test

def find_accuracy(result):
    numtest = result.shape[0]
    sum = 0
    for i in range(numtest):
        if(result.loc[i, 'category'] != result.loc[i, 'classified_label']):
            sum+=1
    return (numtest - sum)/numtest
# print(classified_test.head())

## Split data into train and test randomly

In [11]:
train_per = 0.7 #train test split
#randomize indces, take the first 80% of the indeces and last 20 percent as test
indices = np.random.permutation(df_h.shape[0])
train_ind, test_ind = indices[:int(train_per*df_h.shape[0])], indices[int(train_per*df_h.shape[0]):]
# print(len(train_ind), len(test_ind))

#select the data corresponding to the train and test indices and save into 2 dataframes. Reset index afterwards
train_df, test_df = df_h.loc[train_ind, :], df_h.loc[test_ind, :]
train_df, test_df = train_df.reset_index(drop = True), test_df.reset_index(drop = True)
# train_df.drop('index')

print('Train data is : \n', train_df.head(), ' \nnumber of rows = {}'.format(train_df.shape[0]))
print('Test data is : \n', test_df.head(), ' \nnumber of rows = {}'.format(test_df.shape[0]))


Train data is : 
    age  restbps  chol  category
0   53      133   253         1
1   56      147   397         1
2   43      119   239         1
3   25      111   242         0
4   24      106   197         0  
number of rows = 489
Test data is : 
    age  restbps  chol  category
0   45      140   229         1
1   45      130   214         1
2   41      137   262         0
3   32      111   216         0
4   59      142   328         1  
number of rows = 211


In [12]:
# print((train_df.iloc[1] - train_df.iloc[2])[:-1])


In [14]:
for k in range(1, 25):
    result = KNNclassify(k, train_df, test_df)
    accuracy = find_accuracy(result)
    print('k = {}, acc = {}'.format(k, accuracy))

KeyboardInterrupt: 

In [51]:
class Parzen():
    def __init__(self, v, traindf, testdf):
#         v represents window volume. 
        self.v = v
        self.traindf = self.normalize_data(traindf, ['category'])
        self.testdf = self.normalize_data(testdf, ['category'])

    
    def classify(self):
        df_result =  self.testdf.copy()
        class_lab = np.zeros(self.testdf.shape[0])
        for i in range(self.testdf.shape[0]):
            class_lab[i] = self.get_class(self.testdf.iloc[i])
        
        df_result['classified_label'] = class_lab.tolist()
        return df_result
        
    def get_class(self, datapoint):
        dist_train = self.traindf.copy()
        dist = np.zeros(self.traindf.shape[0])
        for i in range(self.traindf.shape[0]):
            dist[i] = self.get_dist(datapoint, self.traindf.iloc[i])
        dist_train['dist'] = dist.tolist()
        #select rows where the distance is less thn v
        Parzen_train = dist_train.loc[dist_train['dist'] < self.v]
        numones = Parzen_train['category'].sum()
        if(numones>=[Parzen_train.shape[0]]):
            label = 1
        else:
            label = 0
        return label
    
    def normalize_data(self, df, no_change_column_list):
        norm_df = (df - df.min())/(df.max() - df.min())
        for col in no_change_column_list:
            norm_df[col] = df[col]
#         print('-----', norm_df)
        return norm_df

    def get_dist(self, x1, x2):
#         print(np.linalg.norm((x1-x2)[:-1]))
        return np.linalg.norm((x1-x2)[:-1])
        

In [52]:
def Parzenclassify(v, traindf, testdf):
    parzen = Parzen(v, train_df, test_df)
    classified_test = parzen.classify()
    return classified_test

## Split data into train and test randomly

In [53]:
train_per = 0.7# train test split
#randomize indces, take the first 80% of the indeces and last 20 percent as test
indices = np.random.permutation(df_h.shape[0])
train_ind, test_ind = indices[:int(train_per*df_h.shape[0])], indices[int(train_per*df_h.shape[0]):]
# print(len(train_ind), len(test_ind))

#select the data corresponding to the train and test indices and save into 2 dataframes. Reset index afterwards
train_df, test_df = df_h.loc[train_ind, :], df_h.loc[test_ind, :]
train_df, test_df = train_df.reset_index(drop = True), test_df.reset_index(drop = True)
# train_df.drop('index')

# #convert to numpy arrays
# train = train_df.to_numpy()
# test = test_df.to_numpy()

# #split to X test/train and y test/train
# X_train, y_train = train[:, :3], train[:, 3]
# X_test, y_test = test[:, :3], test[:, 3]

# # print(test.shape[0])
# print('X_train_shape = {}'.format(X_train.shape))
# print('y_train_shape = {}'.format(y_train.shape))
# print('X_test_shape = {}'.format(X_test.shape))
# print('y_test_shape = {}'.format(y_test.shape))
print('Train data is : \n', train_df.head(), ' \nnumber of rows = {}'.format(train_df.shape[0]))
print('Test data is : \n', test_df.head(), ' \nnumber of rows = {}'.format(test_df.shape[0]))

Train data is : 
    age  restbps  chol  category
0   26      101   220         0
1   33      116   161         0
2   61      155   343         1
3   34      119   177         0
4   47      111   202         0  
number of rows = 489
Test data is : 
    age  restbps  chol  category
0   49      119   194         1
1   23      105   210         0
2   36      112   259         0
3   67      158   395         1
4   36      133   233         0  
number of rows = 211


In [56]:
train_acc,test_acc = [], []
vol = [i*0.05 for i in range(20)]

In [57]:
for v in vol:
    result = Parzenclassify(v, train_df, test_df)
    accuracy = find_accuracy(result)
#     print('v = {}, acc = {}'.format(v, accuracy))
    test_acc.append(accuracy)

KeyboardInterrupt: 

In [ ]:
for v in vol:
    result = Parzenclassify(v, train_df, train_df)
    accuracy = find_accuracy(result)
    print('v = {}, acc = {}'.format(v, accuracy))
    train_acc.append(accuracy)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_acc, vol, 'b+')
plt.plot(test_acc, vol, 'ro')
plt.show()